# 9-2 Textrank

문장간 유사성이 있는 경우는 connection이 있다고 생각한다..

![대체 텍스트](https://www.researchgate.net/profile/Khushboo_Thakkar3/publication/232645575/figure/fig1/AS:575720050573312@1514273764062/Sample-graph-build-for-sentence-extraction.png)

## 2.1 TextRank 직접 구현하기 (Matrix 활용)

In [7]:
Text = "딸기 바나나 사과 파인애플. 바나나 사과 딸기 포도. 복숭아 수박. 파인애플 사과 딸기 바나나."


In [8]:
from konlpy.tag import Mecab
mecab = Mecab(dicpath = 'C:\mecab\mecab-ko-dic')

In [9]:
sentences = Text.split('. ')

In [10]:
sentences[-1] = sentences[-1][:-1]

In [11]:
sentences

['딸기 바나나 사과 파인애플', '바나나 사과 딸기 포도', '복숭아 수박', '파인애플 사과 딸기 바나나']

In [15]:
sentence_similarity(sentences[0], sentences[2])
(sentences[0], sentences[2])


('딸기 바나나 사과 파인애플', '복숭아 수박')

In [12]:

# 문장간 유사도 측정 (자카드 유사도 사용)
def sentence_similarity(sentence1, sentence2):
    set1 = set(mecab.pos(sentence1))
    set2 = set(mecab.pos(sentence2))
    
    return len(set1 & set2) / len(set1 | set2)
    

sentence_similarity('나는 치킨을 좋아해','나는 치킨을 싫어해')

0.6666666666666666

### 2) 그래프 생성

In [37]:
Text = "딸기 바나나 사과 파인애플 수박. 바나나 사과 딸기 포도. 복숭아 수박. 파인애플 사과 딸기 바나나."
newdoc = sent_tokenize(Text)
newdoc
for i in range(len(newdoc)):
    newdoc[i] = newdoc[i][:-1]
newdoc

['딸기 바나나 사과 파인애플 수박', '바나나 사과 딸기 포도', '복숭아 수박', '파인애플 사과 딸기 바나나']

In [48]:
sentence_similarity(newdoc[0], newdoc[1])

0.5

In [38]:
import numpy as np

In [50]:
def buildMatrix(sentences):
    
    len_sent = len(sentences)

    f_weighted_edge = np.zeros((len_sent, len_sent), np.float32)
    weighted_edge = np.zeros((len_sent, len_sent), np.float32)
    # we must get first score in order to make score
    for i in range(len_sent):
        for j in range(len_sent):
            if not i == j:
                f_weighted_edge[i][j] = sentence_similarity(sentences[i], sentences[j])

    score = f_weighted_edge.sum(axis=1)     
    # print(score)
    # print(weighted_edge)

    for i in range(len_sent):
        for j in range(len_sent):
            
            if not i == j:
                if score[i] == 0:
                    weighted_edge[i][j] = 0
                else:
                    weighted_edge[i][j] = sentence_similarity(sentences[i], sentences[j])/score[i]

    return score, weighted_edge, f_weighted_edge

Text = "딸기 바나나 사과 파인애플 수박. 바나나 사과 딸기 포도. 복숭아 수박. 파인애플 사과 딸기 바나나."
score , weighted_edge, f_weighted_edge = buildMatrix(newdoc)   

In [53]:
f_weighted_edge, score

(array([[0.        , 0.5       , 0.16666667, 0.8       ],
        [0.5       , 0.        , 0.        , 0.6       ],
        [0.16666667, 0.        , 0.        , 0.        ],
        [0.8       , 0.6       , 0.        , 0.        ]], dtype=float32),
 array([1.4666667 , 1.1       , 0.16666667, 1.4000001 ], dtype=float32))

In [54]:
weighted_edge

array([[0.        , 0.3409091 , 0.11363636, 0.54545456],
       [0.45454544, 0.        , 0.        , 0.54545456],
       [1.        , 0.        , 0.        , 0.        ],
       [0.57142854, 0.4285714 , 0.        , 0.        ]], dtype=float32)

In [29]:
score

array([1.6904762, 1.3809524, 0.5714286, 1.6428572], dtype=float32)

In [56]:
0.15 + 0.85*np.dot(weighted_edge.T, score)

array([1.3966666, 1.085    , 0.2916667, 1.34     ], dtype=float32)

### 3) 문장 중요도 계산

In [57]:
def scoring(score, weighted_edge, eps=0.0001, d=0.85, max_iter = 50):
    
    for i in range(max_iter):
        prev_score = np.copy(score)
        score =  (1 - d)  +  d*np.dot(weighted_edge.T, score)
       
        if np.sum(np.fabs(score - prev_score)) <= eps:
            break
                
    return score

In [75]:
gotit = scoring(score, weighted_edge)
np.sort(gotit)[::-1]

array([1.4105095, 1.2790778, 1.0246983, 0.2862456], dtype=float32)

In [76]:
np.argsort(gotit)[::-1]

array([0, 3, 1, 2], dtype=int64)

Object `sort` not found.


### 4) 문서 요약

In [59]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [89]:
from nltk.tokenize import sent_tokenize

def summarize(text, n=10):
    newdoc = sent_tokenize(text)
    
    # 만약 sent_tokenize의 마지막 결과가 .이 포함되면 삭제해줘
    if newdoc[0][-1] == '.':
        for i in range(len(newdoc)):
            newdoc[i] = newdoc[i][:-1]
            
    score, weighted_edge, f_weighted_edge = buildMatrix(newdoc)
    final_score = scoring(score, weighted_edge) # 그럼 이제 최종 스코어랑 문장이랑 짝지어줘서 인덱스를 뽑아와야 함.
    
    largest = np.argsort(final_score)[::-1]
    
    
    
    sent = []
    for i in range(len(newdoc))[:n]:
        sent.append('score : {}, sentences : {}'.format(final_score[largest[i]], newdoc[largest[i]]))
    
    return sent
#     for i in range(len(newdoc))[:n]:
#         print('score : {}, sentences : {}'.format(final_score[largest[i]], newdoc[largest[i]]))
        
        
    
    
    


In [90]:
summary = summarize(Text, 3)

In [91]:
summary

['score : 1.410509467124939, sentences : 딸기 바나나 사과 파인애플 수박',
 'score : 1.2790777683258057, sentences : 파인애플 사과 딸기 바나나',
 'score : 1.024698257446289, sentences : 바나나 사과 딸기 포도']

In [92]:
summary = summarize(Text, 3)
print(' top n important sentences :')
for sent in summary :
    
      print(sent)

 top n important sentences :
score : 1.410509467124939, sentences : 딸기 바나나 사과 파인애플 수박
score : 1.2790777683258057, sentences : 파인애플 사과 딸기 바나나
score : 1.024698257446289, sentences : 바나나 사과 딸기 포도


# 선생님

## 2.1 TextRank 직접 구현하기 (Matrix 활용)

In [ ]:
Text = "딸기 바나나 사과 파인애플. 바나나 사과 딸기 포도. 복숭아 수박. 파인애플 사과 딸기 바나나."
from konlpy.tag import Mecab
mecab = Mecab(dicpath = 'C:\mecab\mecab-ko-dic')

In [99]:

# 문장간 유사도 측정 (자카드 유사도 사용)
def sentence_similarity(sentence1, sentence2):
    mecab = Mecab(dicpath = 'C:\mecab\mecab-ko-dic')
    
    sentence1 =[ word for word in mecab.pos(sentence1) if word[1][0] in ['N', 'V']]
    sentence2 =[ word for word in mecab.pos(sentence2) if word[1][0] in ['N', 'V']]
    
    union = set(sentence1).union(set(sentence2))
    intersection =set(sentence1).intersection(set(sentence2))
    
    return len(intersection) / len(union)

sentence_similarity('나는 치킨을 좋아해','나는 치킨을 싫어해') # 2 / 4

0.5

### 2) 그래프 생성

In [100]:
import numpy as np

def buildMatrix(sentences):
        
        score = np.ones(len(sentences)) # 곱할거니까 받아놔야해..

        weighted_edge = np.zeros((len(sentences), len(sentences)))
        
        for i in range(len(sentences)):
            for j in range(len(sentences)):
                if i == j:
                    continue # 그냥 넘어가~
                weighted_edge[i][j] = sentence_similarity(sentences[i], sentences[j])
                
        for i in range(len(weighted_edge)):
            score[i] = weighted_edge[i].sum()
            weighted_edge[i] /= score[i]
                
                
        return score, weighted_edge

Text = "딸기 바나나 사과 파인애플 수박. 바나나 사과 딸기 포도. 복숭아 수박. 파인애플 사과 딸기 바나나."
buildMatrix(sent_tokenize(Text))    

(array([1.46666667, 1.1       , 0.16666667, 1.4       ]),
 array([[0.        , 0.34090909, 0.11363636, 0.54545455],
        [0.45454545, 0.        , 0.        , 0.54545455],
        [1.        , 0.        , 0.        , 0.        ],
        [0.57142857, 0.42857143, 0.        , 0.        ]]))

### 3) 문장 중요도 계산

In [117]:
def scoring(A, P, eps=0.0001, d=0.85, max_iter = 50):
    
    for iter in range(max_iter):
        
        newP = (1-d) + d* (A.T.dot(P))

       
        if abs((newP - P).sum()) <= eps:
            break
                
    return newP

scoring(weighted_edge, score_init)

NameError: name 'score_init' is not defined

### 4) 문서 요약

In [115]:
from nltk.tokenize import sent_tokenize

def summarize(text, n=10):
    text = sent_tokenize(text)
    score_init, weighted_edge = buildMatrix(text)
    print(score_init, weighted_edge)
    score = scoring(weighted_edge, score_init)
    print(score)
    sorted_score = sorted(enumerate(score), key = lambda item: item[i], reverse = True)[:n]
    return [(text[s[0]], s[1]) for s in sorted_score]
    


In [116]:
summary = summarize(Text, 3)
for sent in summary :
    
      print(sent)

[1.46666667 1.1        0.16666667 1.4       ] [[0.         0.34090909 0.11363636 0.54545455]
 [0.45454545 0.         0.         0.54545455]
 [1.         0.         0.         0.        ]
 [0.57142857 0.42857143 0.         0.        ]]
[1.39666667 1.085      0.29166667 1.34      ]


IndexError: tuple index out of range

In [ ]:
## 이거 변수 넣어서 해보기

## 2.2 TextRank 직접 구현하기 (Graph 활용)

In [118]:
Text = "딸기 바나나 사과 딸기 파인애플. 바나나 사과 딸기. 복숭아 파인애플. 파인애플 사과 딸기 바나나."

### 2) 토큰화

In [119]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [120]:
def sentences(text):
    return sent_tokenize(text)

### 3) 자카드 유사도

- 겹치는 토큰의 비율!!  

근데 그러면 score 필요하잖아.. 그러면 어떻게?

- 문장 1 -> 문장 2   


( 문장 1 - 문장2 유사도) / (문장1 전체 유사도 합(0.5 / (0.5 + 0.8 + 0.167) -> 이거 자체가 score!

유사도를 기반으로, 그냥 초기 스코어를 edge 유사도의 합으로 구해(들어온거)  

그 다음 egge 가중치



In [ ]:
from konlpy.tag import Mecab

# # 문장간 유사도 측정 (자카드 유사도 사용)
# def sentence_similarity(sentence1, sentence2):
#   pass

# sentence_similarity('나는 치킨을 좋아해','나는 치킨을 싫어해')

In [121]:
def connect(nodes): # 인덱스별로 받아온다기 보다 문장 자체를 받아서 하나씩 함
    return [(start,end, sentence_similarity(start,end)) for start in nodes for end in nodes if start is not end]

### 3) 그래프 생성

In [126]:
import networkx as nx

def rank(nodes,edges):
    graph = nx.diamond_graph()
    graph.clear()
    graph.add_nodes_from(nodes)
    graph.add_weighted_edges_from(edges) # 그럼 노드와 엣지가 뭘까> 확인을 해보자
    
    return nx.pagerank(graph)

### 4) 문서 요약

In [128]:
def summarize(text,num_summaries=3):
    nodes = sentences(text)
    print(nodes) # 문장이 나왔고
    edges = connect(nodes)
    scores = rank(nodes, edges)
    print(scores) # 문장에 대한 score가 
    return sorted(scores.items(), key=lambda x: x[1], reverse=True)[:num_summaries]

In [ ]:
#### cf) lambda
현재 구조 score.item()
{ s1 : 0.2, s2 : 0.5, s3 = 0.1} 
보통 딕셔너리는 key가 기준이니, value로 만들어야하니
lambda x 를 가져오면 # 함수로 , 그냥 간편하게 내용만 갖고 있는 함수이다., 아 그래서 x에 들어가는 값은 scores.items() 가 들어감
# 그래서 x[1] 이면 value겠지!

In [129]:
summary=summarize(Text, 3)

for sent in summary :
      print(sent)

['딸기 바나나 사과 딸기 파인애플.', '바나나 사과 딸기.', '복숭아 파인애플.', '파인애플 사과 딸기 바나나.']
{'딸기 바나나 사과 딸기 파인애플.': 0.3270626958069155, '바나나 사과 딸기.': 0.25134837504171265, '복숭아 파인애플.': 0.09452623334445671, '파인애플 사과 딸기 바나나.': 0.3270626958069155}
('딸기 바나나 사과 딸기 파인애플.', 0.3270626958069155)
('파인애플 사과 딸기 바나나.', 0.3270626958069155)
('바나나 사과 딸기.', 0.25134837504171265)


## 2.3 gensim을 활용한 요약

In [130]:
import requests 
from bs4 import BeautifulSoup

def get_news_by_url(url):
    res = requests.get(url)
    bs = BeautifulSoup(res.content, 'html.parser')

    title = bs.select('h3#articleTitle')[0].text #제목
    content = bs.select('#articleBodyContents')[0].get_text().replace('\n', " ") #본문
    content = content.replace("// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}", "")
    return  content.strip()

doc = get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=018&aid=0004430108')
doc[:50]

'과기정통부, 22일 유영민 장관 등 참석해 기념행사2021년까지 1516억원 투입, 510'

In [ ]:
print(docs[1])

In [132]:
from gensim.summarization.summarizer import summarize
summarize(doc).split('.')

['이를 통해 데이터 생태계를 혁신하고 기업의 경쟁력을 제고하는 역할을 수행한다',
 '주요 활용 전략·사례를 보면 빅데이터 활용을 통해 ‘신(新) 시장’을 창출하는 방안을 담고 있다',
 '\n다양한 분석 도구는 물론 인공지능(AI) 학습 알고리즘도 제공해 이용자가 보다 사용하기 편리한 환경을 제공한다',
 '이밖에 필요한 데이터를 쉽게 등록하고 검색할 수 있도록 기준을 마련하고, 데이터 보유와 관리에 대한 체계(거버넌스)를 논의하는 ‘데이터 얼라이언스’를 구성해 보다 안전하게 이용하는 방안도 마련했다',
 '유영민 과기정통부 장관은 “오늘 출범식은 대한민국이 데이터 강국으로 가기 위한 초석을 놓은 자리”라며 “세계 주요국들보다 데이터 경제로 나아가는 발걸음이 다소 늦었지만, 빅데이터 플랫폼과 센터를 지렛대로 우리나라의 낙후된 데이터 생태계를 혁신하고 기업의 경쟁력을 한 단계 제고할 수 있도록 정책적 역량을 집중하겠다”고 밝혔다',
 '이재운 (jwlee@edaily',
 'co',
 'kr)네이버 홈에서 ‘이데일리’ 뉴스 [구독하기▶]꿀잼가득 [영상보기▶] , 청춘뉘우스~ [스냅타임▶]＜ⓒ종합 경제정보 미디어 이데일리 - 무단전재 & 재배포 금지＞']